# Word2vec using Pytorch

This notebook introduces how to implement the NLP technique, so-called word2vec, using Pytorch. This tutorial explains:
1. how to generate the dataset
2. how to build the neural network
3. how to sample data

## The data

Let's introduce the basic concepts:

- *Corpus*: the corpus is the set of texts that define the data set
- *vocabulary*: the set of words in the data set.

For the example, we use the news from the Brown dataset, available on nltk. Non letter characters are removed from the string. Also the text is set in lowercase.

In [1]:
import itertools
import re
import nltk
nltk.download('brown')
from nltk.corpus import brown

corpus = []

for cat in ['romance']:
    for text_id in brown.fileids(cat):
        raw_text = list(itertools.chain.from_iterable(brown.sents(text_id)))
        text = ' '.join(raw_text)
        text = text.lower()
        text.replace('\n', ' ')
        text = re.sub('[^a-z ]+', '', text)
        corpus.append([w for w in text.split() if w != ''])

[nltk_data] Downloading package brown to
[nltk_data]     /Users/rguigoures/nltk_data...
[nltk_data]   Package brown is already up-to-date!


The first step in data preprocessing consists in subsampling frequent words, so that those words are not too often in the neighborhood of the target words. 
Let's call $p_i$ the proportion of word $i$ in the corpus. Then the probability $P(w_i)$ of keeping the word in the corpus is defined as follows:

$$
P(w_i) = \dfrac{10^{-3}}{p_i}\left(\sqrt{10^3 p_i} + 1\right)
$$

In [2]:
from collections import Counter
import random, math

def subsample_frequent_words(corpus):
    filtered_corpus = []
    word_counts = dict(Counter(list(itertools.chain.from_iterable(corpus))))
    sum_word_counts = sum(list(word_counts.values()))
    word_counts = {word: word_counts[word]/float(sum_word_counts) for word in word_counts}
    for text in corpus:
        filtered_corpus.append([])
        for word in text:
            r  = random.random()
            if r < (1+math.sqrt(word_counts[word] * 1e3)) * 1e-3 / float(word_counts[word]):
                filtered_corpus[-1].append(word)
    return filtered_corpus

In [3]:
corpus = subsample_frequent_words(corpus)
vocabulary = set(itertools.chain.from_iterable(corpus))

word_to_index = {w: idx for (idx, w) in enumerate(vocabulary)}
index_to_word = {idx: w for (idx, w) in enumerate(vocabulary)}

Word2vec is a bag of words approach. For each word of the data set, we need to extract the context words, i.e the neighboring words in a certain window of fixed length. For example, in the following sentence:

*My cat is lazy, he sleeps all day long*

For the target word *lazy*, if we consider a window of size 2, then context words are *cat*, *is*, *he* and *sleeps*. 

In [4]:
import numpy as np
import random

context_tuple_list = []
w = 2

for text in corpus:
    for i, word in enumerate(text):
        first_context_word_index = max(0,i-w)
        last_context_word_index = min(i+w, len(text))
        for j in range(first_context_word_index, last_context_word_index):
            if i!=j:
                context_tuple_list.append((word, text[j]))
print("There are {} pairs of target and context words".format(len(context_tuple_list)))

There are 130669 pairs of target and context words


Now we have built the data for the positive examples, i.e the words in the neighborhood of the target word, we need to build a data set with negative examples. The reason for that will be discussed later, when the model will be introduced. For each word in the corpus, the probability of sampling a negative context word is defined as follows:

$$
P(w_i) = \dfrac{\mid w_i \mid^{\frac{3}{4}}}{\displaystyle\sum_{j=1}^n\mid w_j \mid^{\frac{3}{4}}}
$$

In [5]:
from numpy.random import multinomial

negative_tuple_list = []
sample_probability = {}
word_counts = dict(Counter(list(itertools.chain.from_iterable(corpus))))
normalizing_factor = sum([v**0.75 for v in word_counts.values()])
for word in word_counts:
    sample_probability[word] = word_counts[word]**0.75 / normalizing_factor
for word in word_counts:
    sampled_words = multinomial(4*word_counts[word], list(sample_probability.values()))
    for idx, neg_word_count in enumerate(sampled_words):
        for i in range(neg_word_count):
            negative_tuple_list.append((word, list(word_counts.keys())[idx]))
print("{} negative examples have been generated.".format(len(negative_tuple_list)))

174380 negative examples have been generated.


## The Neural network

There two approach of word2vec:

- CBOW (Continuous Bag Of Words). It predicts the target word conditionally to the context. In other words, context words are the input and the target word is the output.
- Skip-gram. It predicts the context conditionally to the target word. In other words, the target word is the input and context words are the output.

The following code is suited for CBOW. 

### Negative examples

Note that the network expects negative examples. The default word2vec algorithm exploits only positive examples and the output function is a softmax. However, using a softmax slows down the learning: softmax is normalized over all the vocabulary, then all the weights of the network are updated at each iteration. Consequently we decide using a sigmoid function as an output instead: only the weights involving the target word are updated. But then the network does not learn from negative examples anymore. That's why we need to input artificially generated negative examples.  

In [6]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import torch.nn.functional as F


class Word2Vec(nn.Module):

    def __init__(self, embedding_size, vocab_size):
        super(Word2Vec, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_size)
        self.linear = nn.Linear(embedding_size, vocab_size)
        
    def forward(self, context_word, neg=False):
        emb = self.embeddings(context_word)
        hidden = self.linear(emb)
        if neg:
            out = F.logsigmoid(-hidden)
        else:
            out = F.logsigmoid(hidden)
        return out

In order to speed up the learning, we propose to use batches. This implies that a bunch of observations are forwarded through the network before doing the backpropagation. Besides being faster, this is also a good way to regularize the parameters of the model.

In [7]:
import random

def get_batches(context_tuple_list, batch_size=100):
    random.shuffle(context_tuple_list)
    batches = []
    batch_X, batch_Y = [], []
    for i in range(len(context_tuple_list)):
        batch_X.append(word_to_index[context_tuple_list[i][0]])
        batch_Y.append(word_to_index[context_tuple_list[i][1]])
        if (i+1) % batch_size == 0 or i == len(context_tuple_list)-1:
            tensor_X = autograd.Variable(torch.from_numpy(np.array(batch_X)).long())
            tensor_Y = autograd.Variable(torch.from_numpy(np.array(batch_Y)).long())
            batches.append((tensor_X, tensor_Y))
            batch_X, batch_Y = [], []
    return batches

Early stopping aims at stopping learning when the loss does not decrease significantly (min_gain parameter) anymore after a certain number of iterations (patience parameter). Early stopping is usually used on the validation loss, but in the case of word2vec, there is no validation since the approach is unsupervised. We apply early stopping on training loss instead.

In [8]:
class EarlyStopping():
    def __init__(self, patience=5, min_gain=0.01):
        self.patience = patience
        self.loss_list = [0]
        self.min_gain = min_gain
        
    def update_loss(self, loss):
        self.loss_list.append(loss)
        if len(self.loss_list) > self.patience:
            del self.loss_list[0]
    
    def stop_training(self):
        if max(self.loss_list) - min(self.loss_list) < self.min_gain:
            return True
        else:
            return False

The data contains 420.000 pairs of words. The neural network in trained with the following parameters:

- embedding size: 200
- batch size: 2000

In [ ]:
import time

vocabulary_size = len(vocabulary)

loss_function = nn.CrossEntropyLoss()
net = Word2Vec(embedding_size=200, vocab_size=vocabulary_size)
optimizer = optim.Adam(net.parameters())
early_stopping = EarlyStopping(patience=5, min_gain=0.001)
context_tensor_list = []

while True:
    losses = []
    context_tuple_batches = get_batches(context_tuple_list, batch_size=2000)
    negative_tuple_batches = get_batches(negative_tuple_list, batch_size=2000)
    for _ in range(len(context_tuple_batches)):
        net.zero_grad()
        target_tensor, context_tensor = context_tuple_batches[i]
        log_probs = net(context_tensor)
        loss = loss_function(log_probs, target_tensor)
        target_tensor, negative_tensor = negative_tuple_batches[i]
        log_probs = net(negative_tensor, neg=True)
        loss += loss_function(log_probs, target_tensor)
        loss.backward()
        optimizer.step()
        losses.append(loss.data)
    print(np.mean(losses))
    early_stopping.update_loss(np.mean(losses))
    if early_stopping.stop_training():
        break

17.989246
17.797407
17.666664
17.56384
17.475521
17.393019
17.302975
17.190355
17.042719
16.857033
16.636396
16.394358
16.148592


Once the network trained, we can use the word embedding and compute the similarity between words. The following function computes the top n closest words for a given word. The similarity used is the cosine.

In [ ]:
import numpy as np

def get_closest_word(word, topn=5):
    word_distance = []
    emb = net.embeddings
    pdist = nn.PairwiseDistance()
    i = word_to_index[word]
    lookup_tensor_i = torch.tensor([i], dtype=torch.long)
    v_i = emb(lookup_tensor_i)
    for j in range(len(vocabulary)):
        if j != i:
            lookup_tensor_j = torch.tensor([j], dtype=torch.long)
            v_j = emb(lookup_tensor_j)
            word_distance.append((index_to_word[j], float(pdist(v_i, v_j))))
    word_distance.sort(key=lambda x: x[1])
    return word_distance[:topn]

In [ ]:
get_closest_word("king")